In [1]:
library(Seurat)
library(tidyverse)
library(ggpubr)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
plot_dir <- paste0(".//kidney_mouse_ntn/reports/plots/cell_type_composition/")
processed_dir <- "./Data/Spatial/visium/kidney/mouse_NTN/processed/ultimate_processed/"

In [3]:
integrated_obj <- readRDS(file.path(processed_dir, "integrated/rds/integrated_annotated_2.rds"))
message("Integrated object read")

Integrated object read



In [ ]:
#obtaining cell_types 
cell_types <- read_csv("./Data/Spatial/visium/kidney/mouse_NTN/processed/deconvolution/split_0/cell_abundance_q5.csv", show_col_types = FALSE) %>%
    colnames() %>%
    .[2:length(.)]

group_variable <- "diagnosis"
alias_id_col <- "alias_id"
Region_col <- "Region2"
Regions <- integrated_obj@meta.data[[Region_col]] %>% unique() %>% as.character()

my_comparisons <- list(c("Healthy", "vehicle"), c("vehicle", "Enalapril"), c("vehicle", "Sema"))
Regions

New names:
• `` -> `...1`


[1] "Glomeruli"     "Cortex"        "ISOM"          "Inner_Medulla"
[5] "OSOM"          "Renal Pelvis"

In [ ]:
data <- integrated_obj@meta.data %>% 
        select(diagnosis, all_of(cell_types), alias_id_col, Region_col) %>%
        mutate(diagnosis = factor(diagnosis, levels = c("Healthy", "vehicle", "Enalapril", "Sema")))

lapply(Regions, function(Region){
    region_data <- filter(data, !!sym(Region_col) == Region)
    
    # Create folder if it doesn't exist
    dir.create(paste0(plot_dir, "/", Region), showWarnings = FALSE)

    lapply(cell_types, function(cell_type){
        cell_type_data <- select(region_data, !!sym(group_variable), all_of(cell_type), !!sym(alias_id_col)) %>%
            group_by(!!sym(group_variable), !!sym(alias_id_col)) %>%
            summarise(cell_type = sum(!!sym(cell_type)), .groups = 'drop')
        
        p <- ggboxplot(cell_type_data, x = group_variable, y = "cell_type",
                    add = "jitter",
                    color = group_variable, # Color by Diagnosis
                    palette = "jco" # jco is a color palette from ggpubr
                    ) +
                    # Remove legend title
                    theme(legend.title = element_blank(),
                    # Remove x labels
                    axis.text.y = element_blank(),
                    axis.ticks.y = element_blank(),
                    #remove line axis
                    axis.line.y = element_blank(),
                    ) +
            stat_compare_means(comparisons = my_comparisons, label = "p.signif") +
            xlab("Treatment") +
            ylab(paste0("Pseudo-bulk cell proportion per spot")) +
            ggtitle(paste0(cell_type, " in ", Region))
        
        ggsave(paste0(plot_dir, "/", Region, "/", cell_type, ".png"), p, width = 10, height = 10, bg = "white", dpi = 300)
    })
})

In [ ]:
# #other approach using facet.by
# p <- ggboxplot(ToothGrowth, x = "supp", y = "len",
#               color = "supp", palette = "npg",
#               add = "jitter",
#               facet.by = "dose", short.panel.labs = FALSE)
# # Use only p.format as label. Remove method name.
# p + stat_compare_means(
#  aes(label = paste0("p = ", after_stat(p.format)))
# )